# **package_LAB**

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots

import package_LAB
from package_LAB import LL_RT
from package_DBR import myRound, SelectPath_RT, Delay_RT, FO_RT, FOPDT, SOPDT, FOPDT_cost, SOPDT_cost, Process, Bode
from importlib import reload
package_LAB = reload(package_LAB)

## LL_RT()

In [77]:
help(LL_RT)

Help on function LL_RT in module package_LAB:

LL_RT(MV, Kp, TLead, TLag, Ts, PV, PVInit=0, method='EBD')
    The function "FO_RT" needs to be included in a "for or while loop".
    
    :MV: input vector
    :Kp: process gain
    :TLead: Lead time constant [s]
    :TLag: Lag time constant [s]
    :Ts: sampling period [s]
    :PV: output vector
    :PVInit: (optional: default value is 0)
    :method: discretisation method (optional: default value is 'EBD')
        EBD: Euler Backward difference
        EFD: Euler Forward difference
        TRAP: Trapezoïdal method
    
    The function "FO_RT" appends a value to the output vector "PV".
    The appended value is obtained from a recurrent equation that depends on the discretisation method.



In [90]:
# Define simulation parameters
TSim = 100
Ts = 0.5
N = int(TSim / Ts) + 1

# Define MV path
MVPath = {0: 0, 5: 1, 50: 2, 80: 3, TSim: 3}

# Define controller parameters
Kp = 1
Tlag = 10
theta = 10

# Initialize lists
t = []
MV = []
MVDelay = []

# Generate time vector and simulate MV
for i in range(0, N):
    t.append(i * Ts)
    SelectPath_RT(MVPath, t, MV)
    Delay_RT(MV, theta, Ts, MVDelay)

# Create figure with subplot
fig = make_subplots(rows=2, cols=1, row_heights=[0.2, 0.8])

# Add MV and MV_Delay traces to subplot
fig.add_trace(
    go.Scatter(
        visible=True,
        line=dict(color="#FF0000", width=3),
        name="MV",
        x=t,
        y=MV), row=1, col=1)

fig.add_trace(
    go.Scatter(
        visible=True,
        line=dict(color="#008000", width=3),
        name="MV_Delay",
        x=t,
        y=MVDelay), row=1, col=1)

# Add PV_EBD_1 traces for different lead times
for step in np.arange(-49, 49, 1):
    
    # Set lead time
    Tlead = step
    
    # Reset lists
    t = []
    MV = []
    MVDelay = []
    PV_EBD_1 = []
    
    # Generate time vector and simulate MV, MV_Delay, and PV_EBD_1
    for i in range(0, N):
        t.append(i * Ts)
        SelectPath_RT(MVPath, t, MV)
        Delay_RT(MV, theta, Ts, MVDelay)
        LL_RT(MVDelay, Kp, Tlead, Tlag, Ts, PV_EBD_1)
    
    # Add PV_EBD_1 trace to subplot
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#00CED1", width=3),
            name="𝜈 = " + str(step),
            x=t,
            y=PV_EBD_1), row=2, col=1)

# Make 10th trace visible initially
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True
    step["args"][0]["visible"][0] = True
    step["args"][0]["visible"][1] = True
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "T_Lead: "},
    pad={"t": 100},
    steps=steps
)]

# Update layout and display figure
fig.update_layout(
    sliders=sliders,
    xaxis_title="Time (s)",
    yaxis_title="PV",
    height=800
)

fig.show()
